In [197]:
using Revise, DynamicHMC, PosteriorDB, Random, StanLogDensityProblems, JSON, BridgeStan, DataFrames, LinearAlgebra, StatsBase, Distributions
using Plots, ColorSchemes, LaTeXStrings, Statistics, CSV, AdvancedHMC, DynamicObjects
Plots.theme(:default)
const BS = BridgeStan;

In [198]:
pdb = PosteriorDB.database()
model = PosteriorDB.model(pdb, "eight_schools_noncentered")
post = PosteriorDB.posterior(pdb, "eight_schools-eight_schools_noncentered")
impl = PosteriorDB.implementation(model, "stan")
path = PosteriorDB.path(impl)
data = PosteriorDB.dataset(post)
s = PosteriorDB.load(data, String);
bs_model = BS.StanModel(stan_file=path, data=s);
ref = PosteriorDB.reference_posterior(post)
df=DataFrame(PosteriorDB.load(ref));

┌ Warning: Loading a shared object '/home/meenaljhajharia/.julia/artifacts/95c3074bd791a2f28857088639b48ca1b786459a/posteriordb-0.4.0/posterior_database/models/stan/eight_schools_noncentered_model.so' which is already loaded.
│ If the file has changed since the last time it was loaded, this load may not update the library!
└ @ BridgeStan /home/meenaljhajharia/.julia/packages/BridgeStan/nCPwJ/src/model.jl:51


In [204]:
using TransformVariables
using TransformedLogDensities, LogDensityProblemsAD
t = as((μ = asℝ, σ = asℝ₊, τ = asℝ₊, θs = as(Array, 8)))
dimension(t)
lp = StanProblem(bs_model);
P = TransformedLogDensity(t, lp)
∇P = ADgradient(:ForwardDiff, lp);

D = 10;
initial_θ = rand(D);
n_samples, n_adapts = 8000, 2000;
metric = DiagEuclideanMetric(D);
hamiltonian = Hamiltonian(metric, lp);
initial_ϵ = find_good_stepsize(hamiltonian, initial_θ);
integrator = Leapfrog(initial_ϵ);
proposal = NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator);
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator));
samples, stats = sample(hamiltonian, proposal, initial_θ, n_samples, adaptor, n_adapts; progress=true);

Sampling  20%|██████▏                        |  ETA: 0:00:00
  iterations:                                   1568
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.01
  n_steps:                                      15
  is_accept:                                    true
  acceptance_rate:                              0.7282397677734759
  log_density:                                  -3.773273371094491
  hamiltonian_energy:                           6.617461851268494
  hamiltonian_energy_error:                     -0.2349030515191739
  max_hamiltonian_energy_error:                 7.2962972774628785
  tree_depth:                                   4
  numerical_error:                              false
  step_size:                                    0.9246504973309816
  nom_step_size:                                0.9246504973309816
  is_adapt:                                     true



















Sampling  48%|██████████████▉                |  ETA: 0:00:00
  iterations:                                   3833
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              1.0
  log_density:                                  -8.078041627623696
  hamiltonian_energy:                           17.595165048117494
  hamiltonian_energy_error:                     -0.1801167651898794
  max_hamiltonian_energy_error:                 -0.1801167651898794
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.4191038129296138
  nom_step_size:                                0.4191038129296138
  is_adapt:                                     false



















Sampling  76%|███████████████████████▌       |  ETA: 0:00:00
  iterations:                                   6064
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              1.0
  log_density:                                  -5.639423623377306
  hamiltonian_energy:                           10.208241445193508
  hamiltonian_energy_error:                     -0.16979272419943037
  max_hamiltonian_energy_error:                 -0.26009244415107524
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.4191038129296138
  nom_step_size:                                0.4191038129296138
  is_adapt:                                     false



















Sampling  98%|██████████████████████████████▌|  ETA: 0:00:00
  iterations:                                   7873
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              0.998288017857404
  log_density:                                  -4.773312238814879
  hamiltonian_energy:                           13.385248943143552
  hamiltonian_energy_error:                     -0.33719257346247566
  max_hamiltonian_energy_error:                 -0.33719257346247566
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.4191038129296138
  nom_step_size:                                0.4191038129296138
  is_adapt:                                     false



















Sampling 100%|███████████████████████████████| Time: 0:00:00
  iterations:                                   8000
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              0.9582116056882012
  log_density:                                  -3.591362139106698
  hamiltonian_energy:                           6.493118776554546
  hamiltonian_energy_error:                     0.018977670436044036
  max_hamiltonian_energy_error:                 0.08845955134173877
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.4191038129296138
  nom_step_size:                                0.4191038129296138
  is_adapt:                                     false
  mass_matrix:                   

┌ Info: Finished 8000 sampling steps for 1 chains in 0.437428684 (s)
│   h = Hamiltonian(metric=DiagEuclideanMetric([0.9429812852230989, 0.7765 ...]), kinetic=GaussianKinetic())
│   κ = HMCKernel{AdvancedHMC.FullMomentumRefreshment, Trajectory{MultinomialTS, Leapfrog{Float64}, GeneralisedNoUTurn{Float64}}}(AdvancedHMC.FullMomentumRefreshment(), Trajectory{MultinomialTS}(integrator=Leapfrog(ϵ=0.419), tc=GeneralisedNoUTurn{Float64}(10, 1000.0)))
│   EBFMI_est = 0.9803837569266837
│   average_acceptance_rate = 0.8770855892701358
└ @ AdvancedHMC /home/meenaljhajharia/.julia/packages/AdvancedHMC/P0nla/src/sampler.jl:246


In [205]:
constrained_draws = hcat([vcat(col...) for col in eachcol(df)]...)
unc_sample_array = vcat([
    param_unconstrain(bs_model, collect(row))' for row in eachrow(constrained_draws)
]...);

In [206]:
lol = hcat(samples...)'
conhmc = vcat([
    param_constrain(bs_model, collect(row))' for row in eachrow(lol)
]...);

In [207]:
mean(unc_sample_array, dims=1)

1×10 Matrix{Float64}:
 6.1505  4.93958  3.90591  4.79602  …  6.31717  4.884  4.41052  0.808081

In [208]:
mean(conhmc, dims=1)

1×10 Matrix{Float64}:
 0.297943  0.10233  -0.0958856  0.0602476  …  0.076262  4.38165  3.56616